In [1]:
import seaborn as sns
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import kendalltau, pearsonr, spearmanr

def spearsonr_pval(x,y):
    return spearmanr(x,y)[1]

data_full = pd.read_csv("LVT-extra-info-M-F-fix-08092022.csv")
data_a = data_full[data_full["Treatmentgroup"].isin(["Group1C","Group3E"])]

data_restore = pd.read_csv("LVT-mass-restored-both.csv")
data_restore_list = data_restore.filename.to_list()
data_restore_list.insert(0, "MouseID")

# this is metadata info, which is extra baylor information.
# careful for the range, [a,b] mean a <= X < b. Also python is 0 based index.
data_M = data_a.iloc[:, np.r_[0, 2:9]]
data_M = data_M.set_index('MouseID')
# this is metabolites, which is features.
data_F = data_a[data_restore_list]
data_F = data_F.set_index('MouseID')

# get the two part of column names: metadata colum name and features column name.
M_column_list = data_M.head(0)
print(M_column_list)
F_column_list = data_F.head(0)
print(F_column_list)

df_col_list = []
df_corr_list = []
df_pval_list = []

for m_col in M_column_list:
    for f_col in F_column_list:
        col_1 = data_M[m_col]
        col_2 = data_F[f_col]
        corr_col_1_2 = col_1.corr(col_2, method='spearman')
        corr_pval_col_1_2 = col_1.corr(col_2, method=spearsonr_pval)
        df_col_list.append(m_col + "-" + f_col)
        df_corr_list.append(corr_col_1_2)
        df_pval_list.append(corr_pval_col_1_2)

print(df_col_list)
print(df_corr_list)
print(df_pval_list)

import statsmodels.api
dr_fdr_pval = statsmodels.stats.multitest.fdrcorrection(df_pval_list, method='indep')
df_final = pd.DataFrame(list(zip(df_col_list, df_corr_list, df_pval_list, dr_fdr_pval[0], dr_fdr_pval[1])), columns=['col_name','corr', 'corr_pval', 'FDR_rejected', 'FDR_pval'])

df_final.to_csv('LVT-no-treat-both-corr-pval.csv', index=False)

Empty DataFrame
Columns: [Heart.Weight..Body.Weight, Liver.Weight..Body.Weight, Ejection.Fraction, PR.Interval..s., QTc..s., P.Amplitude..V., CD3.CD8.IFNg.]
Index: []
Empty DataFrame
Columns: [X141.091_1.54, X296.258_2.976, X366.373_3.699, X608.227_2.768, X708.445_3.006, X709.936_2.18, X776.042_2.398]
Index: []
['Heart.Weight..Body.Weight-X141.091_1.54', 'Heart.Weight..Body.Weight-X296.258_2.976', 'Heart.Weight..Body.Weight-X366.373_3.699', 'Heart.Weight..Body.Weight-X608.227_2.768', 'Heart.Weight..Body.Weight-X708.445_3.006', 'Heart.Weight..Body.Weight-X709.936_2.18', 'Heart.Weight..Body.Weight-X776.042_2.398', 'Liver.Weight..Body.Weight-X141.091_1.54', 'Liver.Weight..Body.Weight-X296.258_2.976', 'Liver.Weight..Body.Weight-X366.373_3.699', 'Liver.Weight..Body.Weight-X608.227_2.768', 'Liver.Weight..Body.Weight-X708.445_3.006', 'Liver.Weight..Body.Weight-X709.936_2.18', 'Liver.Weight..Body.Weight-X776.042_2.398', 'Ejection.Fraction-X141.091_1.54', 'Ejection.Fraction-X296.258_2.976', 'Ej